# Automatic Number Plate Recognition(ANPR)System

Requirement: To develop an efficient ANPR system able to detect 
               and register number plates of vehicles accurately
               in real time
Data: consist of images of vehicles(car,trucks,scooter,etc)

This is just an attempt to replicate the solution ,the data provided in real time is quite different than the one provided

## Existing system

As I could not find any Relevent dataset to emulate this project,I will try to describe it as close to the orignal as possible  

The existing system approached to solve this problem in 2 steps
1. Detect and crop the Number plate on the vehicle with convolutional neural network(Image Localization) 
2. Use of OCR to recognise characters on the vehicle(character segmentation and recognization)

The model uses a simple three layer Convolutional Neural Network with pooling,convolutional layers to train the model and another same model with arbitary window size of 8 by 8 size to detect the number plate

<img src="ANPR images/model.png" height="200" width="200"></img>

### Detecting number plate on the vehicle 

The network uses a sliding window that moves 10 pixels at a time zooms in √2 times at each step and outputs a probability of plate being in that image(shown in green in the above image),while at the same time not generating an excessive number of matches for any single plate.

<img src="ANPR images/window-scan.gif"><div align="center">Figure 1.</div></img>

The text and plate colour are chosen randomly, but the text must be a certain amount darker than the plate.Noise is added at the end to avoid the network from depending too much on sharply defined edges.

once network has detected the plate with confidence, it is then cropped

### Optical Character Recognization(OCR)

#### Preprocessing

Once we get the cropped image,The image is then preprocessed using morphological reconstruction techniques([erosion](https://en.wikipedia.org/wiki/Erosion_(morphology)),[dialton](https://en.wikipedia.org/wiki/Dilation_(morphology)),[top hat transform](https://en.wikipedia.org/wiki/Top-hat_transform)) an then binarized for OCR using [Otsu thresholding](https://en.wikipedia.org/wiki/Otsu%27s_method)

#### OCR

The preprocessed number plate is then segmented into corresponding digits/characters by measuring the region properties of connected components and then using bounding boxes to segment the imkage.Each of this output is then passed into an optical character recongization neural network(same as before 3 convolution layers with relu activation function,maxpooling and a FC layer) where the input is compared with indian number plate(also preprocessed) fonts using softmax function.

### DRAWBACKS

There are several issues with this approach number 
1. plate as there are no standardized characters/numbers or alphabet across vehicles of different states,some even differ in color.

<div style="display:flex">
     <div style="flex:1;padding-right:10px;">
         <img src="ANPR images/number-plate-1.jpg"></img>
    </div>
     <div style="flex:1;padding-left:10px;">
    <img src="ANPR images/number-plate-2.jpg" ></img>        
    </div>

2. Low light,brightness etc have to be adjusted through morphology during preprossing which is not optimal for each case

<div style="display:flex">
<div style="flex:1;padding-right:10px;">
<img src="ANPR images/non_uniform_illuminence_1.jpg"></img>
Figure 2.
</div>
<div style="flex:1;padding-right:10px;">
<img src="ANPR images/non_uniform_illuminence_2.jpg" ></img>        
Figure 3.
</div>
<div style="flex:1;padding-left:10px;">
<img src="ANPR images/non_uniform_illuminence_3.jpg" ></img>    
Figure 4.        
</div>

3. incase of figure 2. if the image is not properly thresholded during binarisation it may have undistinguisable connected components( instead of "u" and "p" being segmented it may be segmented as "up") which may cause the network to falsely detect the plate

4. slow detection model.The system takes several seconds to run on moderately sized image.

## Proposed Solution 

### Image Localization and Segmentation

For Loacalization of image we used [Region-based Fully Convolutional Networks](https://arxiv.org/pdf/1605.06409.pdf) trained on 10000 bounding box labeled images of number plates(for labeling the bounding boxes we used [BBOX-LABEL-TOOL](https://github.com/puzzledqs/BBox-Label-Tool)) 

By using R-FCN we overcame the problem of high-latency as it ook 170ms(on average) to detect a numberplate

### OCR

For this problem we used the previously pretrained CNN network on indian number plate dataset that we had collected and finetuned it with handcrafted feature transformation (SIFT,HOG,Connected Component Analysis) this solved the issues with image preprocessing like connected components,plate disorientation,skewness.

### Parameteric Optimization

here we used [cyclic learnig rates](https://arxiv.org/abs/1506.01186) to determine the best possible weights

### References

1. Figure 1. https://matthewearl.github.io/2016/05/06/cnn-anpr/